# LSA

Latent Semantic Analysis (LSA) is a technique in natural language processing and information retrieval that helps to uncover the hidden (latent) relationships between words in a large corpus of text. It is particularly useful for tasks like document retrieval and topic modeling.

To illustrate the technique, we'll use a simple corpus containing six documents with the following contents:

* d1: "ship ocean wood"
* d2: "boat ocean"
* d3: "ship"
* d4: "wood tree"
* d5: "wood"
* d6: "tree"

Below we build the term-document matrix:

In [1]:
import pandas as pd
import numpy as np

data = {
    'Terms': ['ship', 'boat', 'ocean', 'wood', 'tree'],
    'd1': [1, 0, 1, 1, 0],
    'd2': [0, 1, 1, 0, 0],
    'd3': [1, 0, 0, 0, 0],
    'd4': [0, 0, 0, 1, 1],
    'd5': [0, 0, 0, 1, 0],
    'd6': [0, 0, 0, 0, 1]
}

A = pd.DataFrame(data)

# Set 'Terms' as the index
A.set_index('Terms', inplace=True)

A

,d1,d2,d3,d4,d5,d6
Terms,,,,,,
ship,1,0,1,0,0,0
boat,0,1,0,0,0,0
ocean,1,1,0,0,0,0
wood,1,0,0,1,1,0
tree,0,0,0,1,0,1


Let's say we want to query the corpus for "ship wood". The vector representing the query is built below:

In [2]:
query = pd.Series([1, 0, 0, 1, 0], index=A.index)

query

Terms
ship     1
boat     0
ocean    0
wood     1
tree     0
dtype: int64

Now let's calculate the similarity of each document and the query. We can see that d1 is the most relevant. d2, which contains boat, a concept similar to ship, is not relevant. LSA can help us discover these equivalences and improve our retrieval model. 

In [3]:
# calculate cosine similarity between two vectors
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# calculate cosine similarity between query and each document in A
cosine_similarities = A.T.apply(lambda x: cosine_similarity(x, query), axis=1).sort_values(ascending=False)

cosine_similarities

d1    0.816497
d3    0.707107
d5    0.707107
d4    0.500000
d2    0.000000
d6    0.000000
dtype: float64

## SVD decomposition

The SVD decomposes the term-document matrix into three matrices: U, Σ (Sigma), and V^T. Σ contains the singular values, and U and V^T contain the left and right singular vectors, respectively.

Below we decompose the term-document matrix and print the dimensions of the resulting matrices:

In [4]:
# make a svd decomposition of df

U, s, Vt = np.linalg.svd(A)

# make s a diagonal matrix
sd = np.diag(s)

# pad s with zeros to make it a compatible with V
sd = np.pad(sd, ((0,0),(0, 1)), 'constant' )

print(U.shape)
print(sd.shape)
print(Vt.shape)

(5, 5)
(5, 6)
(6, 6)


Below we display the resulting matrices:

In [5]:
from IPython.display import display, Math
# Function to format matrix for LaTeX display
def matrix_to_latex(matrix):
    return '\\\ '.join(['&'.join([f'{x:.2f}' for x in row]) for row in matrix])

# Create LaTeX strings for each matrix
U_latex = matrix_to_latex(U)
S_latex = matrix_to_latex(sd)
Vt_latex = matrix_to_latex(Vt)

# Display the matrix equation
display(Math(r'A = U\Sigma V^T = \begin{bmatrix}' + U_latex + r'\end{bmatrix} \times \begin{bmatrix}' + S_latex + r'\end{bmatrix} \times \begin{bmatrix}' + Vt_latex + r'\end{bmatrix}'))

<IPython.core.display.Math object>

If we multiply the matrices we get back the original matrix:

In [6]:
# reconstruct the matrix
pd.DataFrame(U @ sd @ Vt, index=A.index, columns=A.columns).round().abs()

,d1,d2,d3,d4,d5,d6
Terms,,,,,,
ship,1.0,0.0,1.0,0.0,0.0,0.0
boat,0.0,1.0,0.0,0.0,0.0,0.0
ocean,1.0,1.0,0.0,0.0,0.0,0.0
wood,1.0,0.0,0.0,1.0,1.0,0.0
tree,0.0,0.0,0.0,1.0,0.0,1.0


## Reducing the dimensionality and recomposing the matrix

One way to introduce the semantic similarity between terms in the term-document matrix is to recompose it with lowed-dimension components. Below we reduce U, Σ, and V^T to two dimensions:

In [7]:
# reconstruct the matrix
r = 2

U_r, s_r, Vt_r = U[:, :r], sd[:r, :r], Vt[:r, :]

s_r

array([[2.16250096, 0.        ],
       [0.        , 1.59438237]])

The reduced matrices are shown below:

In [8]:
# Create LaTeX strings for each matrix
U_latex = matrix_to_latex(U_r)
S_latex = matrix_to_latex(s_r)
Vt_latex = matrix_to_latex(Vt_r)

# Display the matrix equation
display(Math(r'A_r = U_r\Sigma_r V^T_r = \begin{bmatrix}' + U_latex + r'\end{bmatrix} \times \begin{bmatrix}' + S_latex + r'\end{bmatrix} \times \begin{bmatrix}' + Vt_latex + r'\end{bmatrix}'))

<IPython.core.display.Math object>

Now we can recompose the term-document matrix. See how d2 is now related to ship?

In [9]:
A_r = pd.DataFrame(U_r @ s_r @ Vt_r)

# changes the index and columns of A_r to be the same as A
A_r.index = A.index
A_r.columns = A.columns

A_r

,d1,d2,d3,d4,d5,d6
Terms,,,,,,
ship,0.848146,0.515902,0.281625,0.129860,0.205743,-0.075882
boat,0.360778,0.357508,0.155125,-0.205653,-0.025264,-0.180389
ocean,1.003270,0.718285,0.360778,-0.050529,0.155125,-0.205653
wood,0.978006,0.129860,0.205743,1.028534,0.617139,0.411396
tree,0.129860,-0.386042,-0.075882,0.898674,0.411396,0.487278


We can also build a similarity matrix to see related terms:

In [10]:
# make a similarity matrix for all rows of A_r

# Calculate similarity
similarity = A_r.apply(lambda row1: A_r.apply(lambda row2: cosine_similarity(row1, row2), axis=1), axis=1)

similarity

Terms,ship,boat,ocean,wood,tree
Terms,,,,,
ship,1.000000,0.811764,0.978079,0.687557,0.043137
boat,0.811764,1.000000,0.915575,0.134084,-0.548426
ocean,0.978079,0.915575,1.000000,0.521280,-0.165849
wood,0.687557,0.134084,0.521280,1.000000,0.755113
tree,0.043137,-0.548426,-0.165849,0.755113,1.000000


If we run the original query on the updated term-document matrix, we now have d2 as a relevant document:

In [11]:
# calculate cosine similarity between query and each document in A
cosine_similarities = A_r.T.apply(lambda x: cosine_similarity(x, query), axis=1).sort_values(ascending=False)

cosine_similarities

d1    0.767667
d5    0.740680
d3    0.649391
d4    0.590034
d2    0.440244
d6    0.339865
dtype: float64